<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objective" data-toc-modified-id="Objective-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objective</a></span><ul class="toc-item"><li><span><a href="#We-are-trying-to-classify-for-Breast-Cancer-PATH_M_COMBINED:" data-toc-modified-id="We-are-trying-to-classify-for-Breast-Cancer-PATH_M_COMBINED:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>We are trying to classify for Breast Cancer PATH_M_COMBINED:</a></span></li><li><span><a href="#Loading-Pickle-And-CSV-Files-as-Dataframes" data-toc-modified-id="Loading-Pickle-And-CSV-Files-as-Dataframes-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Loading Pickle And CSV Files as Dataframes</a></span></li><li><span><a href="#Custom-Dataset-to-best-utilize-the-data" data-toc-modified-id="Custom-Dataset-to-best-utilize-the-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Custom Dataset to best utilize the data</a></span></li><li><span><a href="#The-Classifier-Structure" data-toc-modified-id="The-Classifier-Structure-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>The Classifier Structure</a></span></li><li><span><a href="#Training-Method" data-toc-modified-id="Training-Method-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Training Method</a></span></li></ul></li></ul></div>

# Objective
We want to use a Convolutional Neural Network to Classify different Cancer Statuses:

 1. M0
 1. M1  


## We are trying to classify for Breast Cancer PATH_M_COMBINED:

In [2]:
#imports
import torch
import torch.nn as nn
from torch.utils.data import dataset, dataloader
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
import pickle
import os
import operator
import sys
from datetime import datetime

## Loading Pickle And CSV Files as Dataframes

In [3]:
#training data
dftraintest = pickle.load(open('keithtest.pkl','rb'))
#validation data
dfval = pickle.load(open('keithval.pkl', 'rb'))
#csv
phenotable_known = pd.read_csv('keith_traintestset_phenotype_primary_M_known.csv')
phenotable_unknown = pd.read_csv('keith_valset_pheonotype_primary_M_unknown.csv')
labels = ['M0', 'M1']

In [4]:
phenotable_known.columns

Index(['sample.id', 'sample.type', 'biopsy.site', 'cancer', 'data.source',
       'gender', 'age', 'read.count.file', 'PATH_T_STAGE', 'PATH_N_STAGE',
       'PATH_M_STAGE', 'stage', 'CANCER_TYPE_ACRONYM', 'PATH_M_combined'],
      dtype='object')

In [5]:
from sklearn.cross_validation import train_test_split

In [6]:
def splitsFun(pheno,target_filter,train_min=10,splits=0.3,seed=0):
    x = dict(pheno[target_filter].value_counts())
    threshold = int(train_min/(1-splits))
    splitsDF = pheno.copy()
    target_passed=[i for i in x if x[i]>=threshold]
    target_failed=[i for i in x if x[i]<threshold]
    splitsDF = splitsDF[splitsDF[target_filter].isin(target_passed)]
    train, test = train_test_split(splitsDF.index, test_size=0.3,
                               random_state = seed ,
                               stratify=splitsDF[target_filter])
    return train, test, splitsDF

In [7]:
train, test, _ = splitsFun(phenotable_known,'PATH_M_combined')

In [8]:
trainpheno = phenotable_known.loc[train]
testpheno = phenotable_known.loc[test]
trainpheno.to_csv('trainpheno.csv')
testpheno.to_csv('testpheno.csv')

In [9]:
dftrain = dftraintest.loc[trainpheno['sample.id']]

In [10]:
dftest = dftraintest.loc[testpheno['sample.id']]

In [11]:
trainpheno.shape, testpheno.shape

((2903, 14), (1245, 14))

In [11]:
phenotable_known.shape

(4148, 14)

In [12]:
phenotable_known

,sample.id,sample.type,biopsy.site,cancer,data.source,gender,age,read.count.file,PATH_T_STAGE,PATH_N_STAGE,PATH_M_STAGE,stage,CANCER_TYPE_ACRONYM,PATH_M_combined
0,TCGA-05-4244-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Male,NaN,TOIL.RData,T2,N2,M1,STAGE IV,LUAD,M1
1,TCGA-05-4250-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Female,NaN,TOIL.RData,T3,N1,M0,STAGE IIIA,LUAD,M0
2,TCGA-05-4382-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Male,NaN,TOIL.RData,T2,N0,M0,STAGE IB,LUAD,M0
3,TCGA-05-4384-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Male,NaN,TOIL.RData,T2,N2,M0,STAGE IIIA,LUAD,M0
4,TCGA-05-4389-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Male,NaN,TOIL.RData,T1,N0,M0,STAGE IA,LUAD,M0
5,TCGA-05-4390-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Female,NaN,TOIL.RData,T2,N0,M0,STAGE IB,LUAD,M0
6,TCGA-05-4395-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Male,NaN,TOIL.RData,T4,N2,M0,STAGE IIIB,LUAD,M0
7,TCGA-05-4396-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Male,NaN,TOIL.RData,T4,N1,M0,STAGE IIIB,LUAD,M0
8,TCGA-05-4397-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Male,NaN,TOIL.RData,T2,N1,M0,STAGE IIB,LUAD,M0
9,TCGA-05-4398-01,primary,LUNG,Lung Adenocarcinoma,TCGA,Female,NaN,TOIL.RData,T4,N3,M0,STAGE IIIB,LUAD,M0


## Custom Dataset to best utilize the data

In [12]:
#reliance on Pandas
class BC_Dataset(torch.utils.data.Dataset):
    
    def __init__(self, pickleframe, phenotable_csv):
        '''ARGS
            pickleframe: Dataframe that was loaded from the pickle dump, this has all the Gene Float Values by sample
            phenotable_csv: CSV File that will be read and 
        '''
        self.traintest = pickleframe
        self.phenotabledf = pd.read_csv(phenotable_csv)
        
    def __len__(self):
        return len(self.traintest)
    
    def __getitem__(self, idx):
        labels = torch.tensor([0,1], dtype=torch.long) if self.phenotabledf['PATH_M_combined'][idx] == 'M1' else torch.tensor([1,0], dtype=torch.long)
        return [labels, torch.tensor(dftraintest.loc[self.phenotabledf['sample.id'][idx], 'ENSG00000000003':'ENSGR0000281849'].values).float()]        

## The Classifier Structure

In [67]:
#A convolutional neural network for this classification
class BC_Classifier(nn.Module):
    def __init__(self):
        super(BC_Classifier, self).__init__()
        self.num_classes = len(labels)
        self.hidden_layer = torch.nn.Sequential(
                        torch.nn.Linear(60498, 1000),
                        torch.nn.ReLU(0.2),
                        torch.nn.Sigmoid(),
                        torch.nn.Linear(1000, 200),
                        #torch.nn.ReLU(0.2,inplace=True),
                        torch.nn.Linear(200,10),
                        #torch.nn.ReLU(0.2,inplace=True),
                        torch.nn.Linear(10, self.num_classes),
                        torch.nn.Sigmoid()
                        
                        )
    
    def forward(self, x):
        x = self.hidden_layer(x)
        return x
    

## Training Method

In [68]:
### HYPER PARAMETERS ###
opt = pd.DataFrame()
opt.batch_size = 128
opt.cuda = False #True if torch.cuda.is_available() else False
opt.lr = 0.001
opt.beta1 = .5
opt.epochs = 50
opt.n_genes = 60498
opt.log_interval = 1

y_dim= 1


#loading data 
train_set = BC_Dataset(pickleframe=dftrain,phenotable_csv='trainpheno.csv')
#train_set_ten = torch.utils.data.TensorDataset(train_set[:1800])
#wrapping this in a dataloader
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=opt.batch_size)

test_set = BC_Dataset(dftest, phenotable_csv='testpheno.csv')
test_loader = torch.utils.data.DataLoader(test_set, shuffle = False, batch_size=opt.batch_size)

x_batch = torch.FloatTensor(opt.batch_size, opt.n_genes)
y_batch = torch.LongTensor(opt.batch_size, y_dim)

x_batch = Variable(x_batch)
y_batch = Variable(y_batch)

#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()

x_batch = torch.FloatTensor(opt.batch_size, opt.n_genes)
y_batch = torch.LongTensor(opt.batch_size, y_dim)

model = BC_Classifier()

if opt.cuda:
    model.cuda()
    criterion.cuda()
    x_batch, y_batch = x_batch.cuda(), y_batch.cuda()
    
x_batch = Variable(x_batch)
y_batch = Variable(y_batch)

optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr)

In [69]:
#criterion = nn.BCEWithLogitsLoss()

In [85]:
def train(model = model, optimizer = optimizer, epochs = opt.epochs):
    for epoch in range(epochs + 1):
        train_epoch_test(epoch,  model, train_loader, optimizer)
        #trian_test_epoch(model, test_loader)
        
#def mse_loss(input, target):
 #   return torch.sum((input - target)^2) / input.data.nelement()

def train_epoch(epoch, model, data_loader, optimizer):
    model.train()
    pid = os.getpid()
    for batch_idx, (targets, data) in enumerate(data_loader):
        running_loss = 0.0
        optimizer.zero_grad()
        x_batch.data.resize_(data.size()).copy_(data)
        y_batch.data.resize_(targets.size()).copy_(targets)
        output = model(x_batch)
        loss = criterion(output, y_batch.float())
        running_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % opt.log_interval == 0:
            print('%s-[%03d/%03d][%04d/%04d]--loss: %6.4f' % (
                datetime.now(), epoch, opt.epochs, batch_idx, len(data_loader), loss.item()))
            sys.stdout.flush()
            
            
def train_epoch_test(epoch, model, data_loader, optimizer):
    model.train()
    #pid = os.getpid()
    for batch_idx, (target, data) in enumerate(data_loader):
        iter_idx = epoch*len(data_loader) + batch_idx
        
        running_loss = 0.0
        optimizer.zero_grad()
        if opt.cuda:
            data, target = Variable(data.cuda()), Variable(target.cuda())
        output = model(data)
        
        loss = criterion(output, target.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        #tensorboard_logger.log_scalar('loss/loss_mse', loss.data[0], iter_idx)
        if batch_idx % opt.log_interval == 0:
            #copied printing from AE training method
            print('%s-[%03d/%03d][%04d/%04d]--loss: %6.4f' % (
                datetime.now(), epoch, opt.epochs, batch_idx, len(data_loader),loss.data[0].item()))
            sys.stdout.flush()

def test_epoch(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data)
            test_loss += F.nll_loss(output[1], target.float(), reduction='sum').item() # sum up batch loss
            pred = output.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target).sum().item()

    test_loss /= len(data_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))
    



print('loaded model')


loaded model


In [ ]:
train(model, optimizer=optimizer, epochs= 100)

2018-11-07 21:28:33.358138-[000/050][0000/0023]--loss: 0.1529


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


2018-11-07 21:28:33.889881-[000/050][0001/0023]--loss: 0.2751
2018-11-07 21:28:34.414906-[000/050][0002/0023]--loss: 0.1511
2018-11-07 21:28:34.944910-[000/050][0003/0023]--loss: 0.2969
2018-11-07 21:28:35.474780-[000/050][0004/0023]--loss: 0.1702
2018-11-07 21:28:35.995148-[000/050][0005/0023]--loss: 0.1909
2018-11-07 21:28:36.476269-[000/050][0006/0023]--loss: 0.2559
2018-11-07 21:28:36.977764-[000/050][0007/0023]--loss: 0.1681
2018-11-07 21:28:37.497446-[000/050][0008/0023]--loss: 0.2345
2018-11-07 21:28:37.998099-[000/050][0009/0023]--loss: 0.2348
2018-11-07 21:28:38.499164-[000/050][0010/0023]--loss: 0.2350
2018-11-07 21:28:38.999815-[000/050][0011/0023]--loss: 0.3033
2018-11-07 21:28:39.501555-[000/050][0012/0023]--loss: 0.1444
2018-11-07 21:28:40.003470-[000/050][0013/0023]--loss: 0.2347
2018-11-07 21:28:40.505011-[000/050][0014/0023]--loss: 0.2121
2018-11-07 21:28:41.006381-[000/050][0015/0023]--loss: 0.3688
2018-11-07 21:28:41.507606-[000/050][0016/0023]--loss: 0.2121
2018-11-

2018-11-07 21:29:41.931210-[005/050][0019/0023]--loss: 0.1900
2018-11-07 21:29:42.453082-[005/050][0020/0023]--loss: 0.2121
2018-11-07 21:29:42.975434-[005/050][0021/0023]--loss: 0.1902
2018-11-07 21:29:43.439878-[005/050][0022/0023]--loss: 0.1877
2018-11-07 21:29:43.962671-[006/050][0000/0023]--loss: 0.2341
2018-11-07 21:29:44.486426-[006/050][0001/0023]--loss: 0.2342
2018-11-07 21:29:45.017131-[006/050][0002/0023]--loss: 0.1680
2018-11-07 21:29:45.547789-[006/050][0003/0023]--loss: 0.3671
2018-11-07 21:29:46.077941-[006/050][0004/0023]--loss: 0.1903
2018-11-07 21:29:46.587695-[006/050][0005/0023]--loss: 0.1471
2018-11-07 21:29:47.119032-[006/050][0006/0023]--loss: 0.2340
2018-11-07 21:29:47.650300-[006/050][0007/0023]--loss: 0.1905
2018-11-07 21:29:48.182392-[006/050][0008/0023]--loss: 0.2340
2018-11-07 21:29:48.713098-[006/050][0009/0023]--loss: 0.2340
2018-11-07 21:29:49.217142-[006/050][0010/0023]--loss: 0.1903
2018-11-07 21:29:49.736430-[006/050][0011/0023]--loss: 0.2997
2018-11-

2018-11-07 21:30:50.199897-[011/050][0014/0023]--loss: 0.0818
2018-11-07 21:30:50.713154-[011/050][0015/0023]--loss: 0.2783
2018-11-07 21:30:51.243687-[011/050][0016/0023]--loss: 0.1899
2018-11-07 21:30:51.764921-[011/050][0017/0023]--loss: 0.3016
2018-11-07 21:30:52.283497-[011/050][0018/0023]--loss: 0.2344
2018-11-07 21:30:52.807640-[011/050][0019/0023]--loss: 0.2121
2018-11-07 21:30:53.332209-[011/050][0020/0023]--loss: 0.2565
2018-11-07 21:30:53.866597-[011/050][0021/0023]--loss: 0.1901
2018-11-07 21:30:54.317435-[011/050][0022/0023]--loss: 0.1877
2018-11-07 21:30:54.823394-[012/050][0000/0023]--loss: 0.2998
2018-11-07 21:30:55.346921-[012/050][0001/0023]--loss: 0.1688
2018-11-07 21:30:55.869649-[012/050][0002/0023]--loss: 0.4073
2018-11-07 21:30:56.391668-[012/050][0003/0023]--loss: 0.2126
2018-11-07 21:30:56.912176-[012/050][0004/0023]--loss: 0.2547
2018-11-07 21:30:57.433588-[012/050][0005/0023]--loss: 0.1109
2018-11-07 21:30:57.954723-[012/050][0006/0023]--loss: 0.2340
2018-11-

...

2018-11-07 21:43:11.249806-[075/050][0005/0023]--loss: 0.2121
2018-11-07 21:43:11.750772-[075/050][0006/0023]--loss: 0.2347
2018-11-07 21:43:12.250607-[075/050][0007/0023]--loss: 0.1668
2018-11-07 21:43:12.752543-[075/050][0008/0023]--loss: 0.2350
2018-11-07 21:43:13.255405-[075/050][0009/0023]--loss: 0.3265
2018-11-07 21:43:13.758661-[075/050][0010/0023]--loss: 0.2348
2018-11-07 21:43:14.261276-[075/050][0011/0023]--loss: 0.2568
2018-11-07 21:43:14.731464-[075/050][0012/0023]--loss: 0.2560
2018-11-07 21:43:15.251103-[075/050][0013/0023]--loss: 0.1695
2018-11-07 21:43:15.764885-[075/050][0014/0023]--loss: 0.2550
2018-11-07 21:43:16.279493-[075/050][0015/0023]--loss: 0.1503
2018-11-07 21:43:16.792608-[075/050][0016/0023]--loss: 0.1921
2018-11-07 21:43:17.306136-[075/050][0017/0023]--loss: 0.1917
2018-11-07 21:43:17.819614-[075/050][0018/0023]--loss: 0.2764
2018-11-07 21:43:18.325821-[075/050][0019/0023]--loss: 0.2124
2018-11-07 21:43:18.802142-[075/050][0020/0023]--loss: 0.1477
2018-11-

2018-11-07 21:44:17.519493-[081/050][0000/0023]--loss: 0.1186
2018-11-07 21:44:18.031749-[081/050][0001/0023]--loss: 0.2375
2018-11-07 21:44:18.548815-[081/050][0002/0023]--loss: 0.1895
2018-11-07 21:44:19.061407-[081/050][0003/0023]--loss: 0.3128
2018-11-07 21:44:19.572071-[081/050][0004/0023]--loss: 0.3114
2018-11-07 21:44:20.096519-[081/050][0005/0023]--loss: 0.1894
2018-11-07 21:44:20.622241-[081/050][0006/0023]--loss: 0.2593
2018-11-07 21:44:21.140951-[081/050][0007/0023]--loss: 0.1897
2018-11-07 21:44:21.659287-[081/050][0008/0023]--loss: 0.1900
2018-11-07 21:44:22.176070-[081/050][0009/0023]--loss: 0.2343
2018-11-07 21:44:22.676788-[081/050][0010/0023]--loss: 0.2776
2018-11-07 21:44:23.177639-[081/050][0011/0023]--loss: 0.1266
2018-11-07 21:44:23.677760-[081/050][0012/0023]--loss: 0.2552
2018-11-07 21:44:24.178590-[081/050][0013/0023]--loss: 0.1912
2018-11-07 21:44:24.679350-[081/050][0014/0023]--loss: 0.2339
2018-11-07 21:44:25.179842-[081/050][0015/0023]--loss: 0.3195
2018-11-

2018-11-07 21:45:24.624746-[086/050][0018/0023]--loss: 0.2574
2018-11-07 21:45:25.144226-[086/050][0019/0023]--loss: 0.2802
2018-11-07 21:45:25.666984-[086/050][0020/0023]--loss: 0.1896
2018-11-07 21:45:26.187023-[086/050][0021/0023]--loss: 0.2121
2018-11-07 21:45:26.631610-[086/050][0022/0023]--loss: 0.2862
2018-11-07 21:45:27.153099-[087/050][0000/0023]--loss: 0.3011
2018-11-07 21:45:27.673201-[087/050][0001/0023]--loss: 0.3216
2018-11-07 21:45:28.192401-[087/050][0002/0023]--loss: 0.2553
2018-11-07 21:45:28.712971-[087/050][0003/0023]--loss: 0.1922
2018-11-07 21:45:29.232417-[087/050][0004/0023]--loss: 0.2546
2018-11-07 21:45:29.751921-[087/050][0005/0023]--loss: 0.1738
2018-11-07 21:45:30.271501-[087/050][0006/0023]--loss: 0.1542
2018-11-07 21:45:30.790939-[087/050][0007/0023]--loss: 0.2142
2018-11-07 21:45:31.311199-[087/050][0008/0023]--loss: 0.1936
2018-11-07 21:45:31.831069-[087/050][0009/0023]--loss: 0.1518
2018-11-07 21:45:32.350683-[087/050][0010/0023]--loss: 0.2339
2018-11-

2018-11-07 21:46:32.161610-[092/050][0013/0023]--loss: 0.3000
2018-11-07 21:46:32.681373-[092/050][0014/0023]--loss: 0.2122
2018-11-07 21:46:33.200676-[092/050][0015/0023]--loss: 0.1460
2018-11-07 21:46:33.720118-[092/050][0016/0023]--loss: 0.1899
2018-11-07 21:46:34.238691-[092/050][0017/0023]--loss: 0.3020
2018-11-07 21:46:34.711133-[092/050][0018/0023]--loss: 0.3020
2018-11-07 21:46:35.234095-[092/050][0019/0023]--loss: 0.2789
2018-11-07 21:46:35.752154-[092/050][0020/0023]--loss: 0.1902
2018-11-07 21:46:36.283794-[092/050][0021/0023]--loss: 0.2122
2018-11-07 21:46:36.740749-[092/050][0022/0023]--loss: 0.1564
2018-11-07 21:46:37.273896-[093/050][0000/0023]--loss: 0.2772
2018-11-07 21:46:37.805532-[093/050][0001/0023]--loss: 0.1479
2018-11-07 21:46:38.334757-[093/050][0002/0023]--loss: 0.2986
2018-11-07 21:46:38.857053-[093/050][0003/0023]--loss: 0.2339
2018-11-07 21:46:39.367974-[093/050][0004/0023]--loss: 0.2124
2018-11-07 21:46:39.896168-[093/050][0005/0023]--loss: 0.2551
2018-11-

2018-11-07 21:47:40.301646-[098/050][0008/0023]--loss: 0.1425
2018-11-07 21:47:40.823347-[098/050][0009/0023]--loss: 0.1892
2018-11-07 21:47:41.346320-[098/050][0010/0023]--loss: 0.3334
2018-11-07 21:47:41.875074-[098/050][0011/0023]--loss: 0.1653
2018-11-07 21:47:42.398181-[098/050][0012/0023]--loss: 0.2613
2018-11-07 21:47:42.921120-[098/050][0013/0023]--loss: 0.1892
2018-11-07 21:47:43.442491-[098/050][0014/0023]--loss: 0.3315
2018-11-07 21:47:43.969929-[098/050][0015/0023]--loss: 0.1892
2018-11-07 21:47:44.498768-[098/050][0016/0023]--loss: 0.2581
2018-11-07 21:47:45.031827-[098/050][0017/0023]--loss: 0.2571
2018-11-07 21:47:45.563675-[098/050][0018/0023]--loss: 0.1902
2018-11-07 21:47:46.084886-[098/050][0019/0023]--loss: 0.3204
2018-11-07 21:47:46.611560-[098/050][0020/0023]--loss: 0.2550
2018-11-07 21:47:47.146847-[098/050][0021/0023]--loss: 0.2134
2018-11-07 21:47:47.609123-[098/050][0022/0023]--loss: 0.2213
2018-11-07 21:47:48.138164-[099/050][0000/0023]--loss: 0.2346
2018-11-

In [19]:
torch.cuda.empty_cache()